# Extract Visual Query

In [1]:
# setup config
import os
from PIL import Image
import torch
from maskrcnn_benchmark.data import make_data_loader
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from maskrcnn_benchmark.data import transforms as T
from maskrcnn_benchmark.config import try_to_find
from maskrcnn_benchmark.structures.bounding_box import BoxList
from maskrcnn_benchmark.utils.miscellaneous import mkdir, save_config
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.data.transforms import build_transforms # input image and box transforms

/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:313: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:378: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:428: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/set_loss.py:144: FutureWarning: `torch.cuda.amp.custom_f

### setup config

In [2]:
from maskrcnn_benchmark.config import cfg as cfg_extract_vq
# set other args
cfg_extract_vq.local_rank = 0
cfg_extract_vq.num_gpus = 1
config_file = "configs/pretrain/mq-glip-l.yaml"
cfg_extract_vq.merge_from_file(config_file)
additional_model_config = "configs/vision_query_5shot/lvis_minival.yaml"
cfg_extract_vq.merge_from_file(additional_model_config)

opts = [
    "VISION_QUERY.QUERY_BANK_SAVE_PATH", "MODEL/lvis_query_5_pool7_sel_large.pth",
    "VISION_QUERY.MAX_QUERY_NUMBER", 5,
    "DATASETS.FEW_SHOT", 5, 
    "VISION_QUERY.QUERY_BANK_PATH", "",
    "DATALOADER.NUM_WORKERS", 0
]
cfg_extract_vq.merge_from_list(opts)

cfg_extract_vq.freeze()

# save_config(cfg_extract_vq, "visualization/lvis_extract_visual_query_config_from_notebook.yaml")

In [3]:
cfg = cfg_extract_vq

### build model

In [4]:
# modified from tools.train_net.extract_query
if cfg.DATASETS.FEW_SHOT:
    assert cfg.DATASETS.FEW_SHOT == cfg.VISION_QUERY.MAX_QUERY_NUMBER, 'To extract the right query instances, set VISION_QUERY.MAX_QUERY_NUMBER = DATASETS.FEW_SHOT.'

model = build_detection_model(cfg)
device = torch.device(cfg.MODEL.DEVICE)
model.to(device)
model.eval()

/home/jielei/miniconda/envs/ov_det/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


VISION BACKBONE USE GRADIENT CHECKPOINTING:  False
LANGUAGE BACKBONE USE GRADIENT CHECKPOINTING:  False


Some weights of QVBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['encoder.qv_layer.0.attn.norm.bias', 'encoder.qv_layer.0.attn.norm.weight', 'encoder.qv_layer.0.attn.norm_kv.bias', 'encoder.qv_layer.0.attn.norm_kv.weight', 'encoder.qv_layer.0.attn.to_kv.weight', 'encoder.qv_layer.0.attn.to_out.weight', 'encoder.qv_layer.0.attn.to_q.weight', 'encoder.qv_layer.0.attn_gate.linear1.weight', 'encoder.qv_layer.0.attn_gate.linear2.weight', 'encoder.qv_layer.0.attn_gate.norm.bias', 'encoder.qv_layer.0.attn_gate.norm.weight', 'encoder.qv_layer.0.ff.linear1.weight', 'encoder.qv_layer.0.ff.linear2.weight', 'encoder.qv_layer.0.ff.norm.bias', 'encoder.qv_layer.0.ff.norm.weight', 'encoder.qv_layer.0.ff_gate', 'encoder.qv_layer.1.attn.norm.bias', 'encoder.qv_layer.1.attn.norm.weight', 'encoder.qv_layer.1.attn.norm_kv.bias', 'encoder.qv_layer.1.attn.norm_kv.weight', 'encoder.qv_layer.1.attn.to_kv.weight', 'encoder.qv_layer.1.attn.to_out.weight

EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B


### build dataloader

In [5]:
data_loader = make_data_loader(
    cfg,
    is_train=False,
    is_cache=True,
    is_distributed= cfg.num_gpus > 1,
)
assert isinstance(data_loader, list) and len(data_loader)==1
data_loader = data_loader[0]

When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14


The combined datasets are: ('lvis_grounding_train_for_obj365',).
loading annotations into memory...
Done (t=23.57s)
creating index...
index created!
lvis_grounding_train_for_obj365 has the 3997 data points CocoGrounding_New


In [129]:
data_loader.dataset

Dataset CocoGrounding_New
    Number of datapoints: 3997
    Root location: ./DATASET/coco

In [6]:
print(f"cfg.SOLVER.IMS_PER_BATCH: {cfg.SOLVER.IMS_PER_BATCH}")
print(f"data_loader.batch_size: {data_loader.batch_size}")
print(f"len(data_loader): {len(data_loader)}")
print("-"*30)
print(f"cfg.DATASETS:\n{cfg.DATASETS}")

cfg.SOLVER.IMS_PER_BATCH: 8
data_loader.batch_size: None
len(data_loader): 500
------------------------------
cfg.DATASETS:
ADD_DET_PROMPT: False
ADD_DET_PROMPT_ADVANCED: False
ADD_NORMED_CXCY: False
ALTERNATIVE_TRAINING: False
BING_INDEX_LIST: []
BOX_THRESHOLD: 0.1
CAPTION_CONF: 0.5
CAPTION_FORMAT_VERSION: v2
CAPTION_MIN_BOX: 1
CAPTION_NMS: -1.0
CAPTION_PROMPT: None
CLASS_AGNOSTIC: False
CLASS_CONCAT: False
COCO_COPY: 1
CONTROL_PROB: (0.0, 0.0, 0.5, 0.0)
DETECT_NONEXIST: False
DISABLE_CLIP_TO_IMAGE: False
DISABLE_SHUFFLE: False
DIVER_BOX_FOR_VQA: False
EXCLUDE_CROWD: True
FEW_SHOT: 5
FLICKR_COPY: 8
FLICKR_GT_TYPE: separate
FULL_QUESTION_PROB: 0.5
FURTHER_SCREEN: True
GENERAL_COPY: -1
GENERAL_COPY_TEST: -1
INFERENCE_CAPTION: False
IN_COPY: 2
LOCAL_DEBUG: False
LVIS_COPY: 1
LVIS_USE_NORMAL_AP: False
MAX_BOX: -1
MIXED_COPY: 4
MULTISTAGE_TRAINING: False
NEG_QUESTION_PROB: 0.8
NO_MINUS_ONE_FOR_ONE_HOT: False
NO_RANDOM_PACK_PROBABILITY: 0.4
OBJECT365_COPY: 2
OI_COPY: 1
ONE_HOT: False
OVERRI

In [87]:
cfg.DATALOADER

CfgNode({'NUM_WORKERS': 0, 'SIZE_DIVISIBILITY': 32, 'ASPECT_RATIO_GROUPING': False, 'MIN_KPS_PER_IMS': 0, 'USE_RANDOM_SEED': False, 'DISTRIBUTE_CHUNK_AMONG_NODE': False})

#### check data

In [7]:
# Get a single batch
batch_iterator = iter(data_loader)
batch_data = next(batch_iterator)
print("Batch data:", batch_data)

Batch data: (<maskrcnn_benchmark.structures.image_list.ImageList object at 0x7d468fc7a2f0>, (BoxList(num_boxes=2, image_width=1196, image_height=800, mode=xyxy), BoxList(num_boxes=1, image_width=1204, image_height=800, mode=xyxy), BoxList(num_boxes=6, image_width=800, image_height=1064, mode=xyxy), BoxList(num_boxes=6, image_width=1071, image_height=800, mode=xyxy), BoxList(num_boxes=14, image_width=800, image_height=1049, mode=xyxy), BoxList(num_boxes=3, image_width=1049, image_height=800, mode=xyxy), BoxList(num_boxes=3, image_width=800, image_height=1066, mode=xyxy), BoxList(num_boxes=1, image_width=1201, image_height=800, mode=xyxy)), (0, 1, 2, 3, 4, 5, 6, 7), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), None, None)


In [8]:
images, targets, *_ = batch_data

 1. check images

In [9]:
print(f"repr(images):\n{repr(images)}")
print("-"*30)
print(f"images.tensors.shape:\n{images.tensors.shape}")
print("-"*30)
print(f"images.image_sizes:\n{images.image_sizes}")

repr(images):
------------------------------
images.tensors.shape:
torch.Size([8, 3, 1088, 1216])
------------------------------
images.image_sizes:
[torch.Size([800, 1196]), torch.Size([800, 1204]), torch.Size([1064, 800]), torch.Size([800, 1071]), torch.Size([1049, 800]), torch.Size([800, 1049]), torch.Size([1066, 800]), torch.Size([800, 1201])]


2. check targets (boxes)

In [10]:
print(f"repr(targets):\n{repr(targets)}")
print("-"*30)
print(f"repr(targets[0]):\n{repr(targets[0])}")
print("-"*30)
print(f"targets[0].bbox:\n{targets[0].bbox}")
print("-"*30)
print(f"targets[0].size:\n{targets[0].size}")
print("-"*30)
print(f"targets[0].fields():\n{targets[0].fields()}")
print("-"*30)
print(f"targets[0].get_field('labels'):\n{targets[0].get_field('labels')}")

repr(targets):
(BoxList(num_boxes=2, image_width=1196, image_height=800, mode=xyxy), BoxList(num_boxes=1, image_width=1204, image_height=800, mode=xyxy), BoxList(num_boxes=6, image_width=800, image_height=1064, mode=xyxy), BoxList(num_boxes=6, image_width=1071, image_height=800, mode=xyxy), BoxList(num_boxes=14, image_width=800, image_height=1049, mode=xyxy), BoxList(num_boxes=3, image_width=1049, image_height=800, mode=xyxy), BoxList(num_boxes=3, image_width=800, image_height=1066, mode=xyxy), BoxList(num_boxes=1, image_width=1201, image_height=800, mode=xyxy))
------------------------------
repr(targets[0]):
BoxList(num_boxes=2, image_width=1196, image_height=800, mode=xyxy)
------------------------------
targets[0].bbox:
tensor([[380.9447,  64.7477, 855.8875, 413.8505],
        [451.7142, 289.8131, 760.7495, 653.2710]])
------------------------------
targets[0].size:
(1196, 800)
------------------------------
targets[0].fields():
['labels', 'boxes', 'caption', 'image_id', 'tokens_po

### Run inference -- use internal function

In [30]:
query_images=defaultdict(list)
query_images = model.extract_query(images.to(device), targets, query_images)

In [35]:
print(f"query_images.keys() {query_images.keys()}")
print(f"query_images[461].shape {query_images[461].shape}")
print(f"query_images[461][0, 0, :10]: {query_images[461][0, 0, :10]}")
print(f"len(query_images): {len(query_images)}")
print("-"*30)
for k, v in query_images.items():
    print(f"class {k}, v.shape {v.shape}")

query_images.keys() dict_keys([461, 1139, 1202, 715, 959, 1045, 547, 1023, 911, 298, 837, 43, 900, 1142, 1017, 544, 422, 800, 876, 1056, 192])
query_images[461].shape torch.Size([1, 1, 256])
query_images[461][0, 0, :10]: tensor([ 0.5390, -0.6056,  0.0860, -0.1750, -1.6917,  0.1566, -0.6254,  0.2460,
         0.3537,  1.0738], device='cuda:0')
len(query_images): 21
------------------------------
class 461, v.shape torch.Size([1, 1, 256])
class 1139, v.shape torch.Size([1, 1, 256])
class 1202, v.shape torch.Size([1, 1, 256])
class 715, v.shape torch.Size([1, 1, 256])
class 959, v.shape torch.Size([1, 1, 256])
class 1045, v.shape torch.Size([1, 1, 256])
class 547, v.shape torch.Size([1, 1, 256])
class 1023, v.shape torch.Size([2, 1, 256])
class 911, v.shape torch.Size([5, 1, 256])
class 298, v.shape torch.Size([3, 1, 256])
class 837, v.shape torch.Size([1, 1, 256])
class 43, v.shape torch.Size([3, 1, 256])
class 900, v.shape torch.Size([2, 1, 256])
class 1142, v.shape torch.Size([3, 1, 25

### Run inference
modified from `model.extract_query` from the cell above

In [13]:
import copy
raw_targets = copy.deepcopy(targets)

In [14]:
images = images.to(device)
targets = [target.to(device)
            for target in targets if target is not None]

In [15]:
images

In [19]:
with torch.no_grad():
    images = to_image_list(images)
    assert 'vl' not in cfg.MODEL.SWINT.VERSION, 'Only support vision inputs now'
    visual_features = model.backbone(images.tensors)    
    query_feats=model.pooler(visual_features, targets) # num_boxes, num_channels, pooler_size, pooler_size
    query_feats=query_feats[None, ...] # 1, num_boxes, num_channels, pooler_size, pooler_size
    query_feats = query_feats.mean(dim=[-2,-1]).permute(1, 0, 2) # num_boxes, num_scales, num_channels
    labels=torch.cat([t.get_field('labels') for t in targets])
    assert len(labels)==len(query_feats)
    max_query_number = cfg.VISION_QUERY.MAX_QUERY_NUMBER

In [128]:
print(f"query_feats.shape: {query_feats.shape}")
print(f"len(labels): {len(labels)}")
print(f"labels {labels}")
print(f"type(visual_features): {type(visual_features)}")
print(f"visual_features[0].shape: {visual_features[0].shape}")

query_feats.shape: torch.Size([36, 1, 256])
len(labels): 36
labels tensor([ 461, 1139, 1202,  715,  959, 1045,  547, 1023, 1023,  911,  911,  911,
         911,  911,  911,  298,  298,  298,  837,   43,   43,   43,  900,  900,
        1142, 1142, 1142, 1017, 1017,  544,  422,  422,  800,  876, 1056,  192],
       device='cuda:0')
type(visual_features): <class 'tuple'>
visual_features[0].shape: torch.Size([8, 256, 136, 152])


### Run inference with loaded single image

##### Load and transform image and boxes

In [70]:
def build_eval_transform(cfg):
    min_size = cfg.INPUT.MIN_SIZE_TEST
    max_size = cfg.INPUT.MAX_SIZE_TEST
    flip_horizontal_prob = 0.0
    fix_res = cfg.INPUT.FIX_RES # false
    if cfg.INPUT.FORMAT != '':
        input_format = cfg.INPUT.FORMAT
    elif cfg.INPUT.TO_BGR255:
        input_format = 'bgr255'
    normalize_transform = T.Normalize(
        mean=cfg.INPUT.PIXEL_MEAN, std=cfg.INPUT.PIXEL_STD, format=input_format
    )
    transform = T.Compose(
        [
            T.Resize(min_size, max_size, restrict=fix_res),
            T.RandomHorizontalFlip(flip_horizontal_prob),
            T.ToTensor(),
            normalize_transform,
        ]
    )
    return transform

In [71]:
eval_transform = build_eval_transform(cfg)

In [196]:
#### load image from path, load boxes
image_path = os.path.join(
    data_loader.dataset.root, 
    data_loader.dataset.coco.loadImgs(data_id)[0]["file_name"]
)
loaded_image = Image.open(image_path).convert("RGB")
# loaded_boxes = dxata_loader.dataset.coco.loadAnns(data_loader.dataset.coco.getAnnIds(data_id))
# tgt = [obj for obj in loaded_boxes if obj["iscrowd"] == 0]
# boxes = [obj["bbox"] for obj in tgt]
# data from the above commented code
loaded_boxes = [
    [203.85, 34.64, 255.15, 187.77],
    [241.72, 155.05, 166.37, 195.45]
]
loaded_boxes = torch.as_tensor(loaded_boxes).reshape(-1, 4)  # guard against no boxes
loaded_target = BoxList(loaded_boxes, loaded_image.size, mode="xywh").convert("xyxy")

anno = data_loader.dataset.coco.loadAnns(data_loader.dataset.coco.getAnnIds(data_id))
labels = torch.tensor([e["category_id"] for e in anno])
loaded_target.add_field("labels", labels)

#### transform
t_image, t_target = eval_transform(loaded_image, loaded_target)
t_image = t_image.to(device)
t_image_list = to_image_list(tensors=t_image[None, :], size_divisible=cfg.DATALOADER.SIZE_DIVISIBILITY)
t_target = [t_target.to(device), ]

In [126]:
t_target[0].bbox

tensor([[380.9447,  64.7477, 855.8875, 413.8505],
        [451.7142, 289.8131, 760.7495, 653.2710]], device='cuda:0')

In [197]:
t_image

tensor([[[-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
         [-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
         [-1.2990, -1.2990, -1.2990,  ..., -0.9156, -0.9330, -0.9330],
         ...,
         [ 1.2980,  1.2980,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
         [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
         [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897]],

        [[-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
         [-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
         [-1.4580, -1.4580, -1.4580,  ..., -0.3725, -0.3550, -0.3550],
         ...,
         [ 1.0980,  1.0980,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
         [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
         [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206]],

        [[-1.6555, -1.6555, -1.6384,  ..., -0.9877, -0.9705, -0.9705],
         [-1.6555, -1.6555, -1.6384,  ..., -0

##### Check against direct loading from data_loader.dataset
Image and boxes match exactly.

In [75]:
data_loader.dataset[0]

(tensor([[[-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
          [-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
          [-1.2990, -1.2990, -1.2990,  ..., -0.9156, -0.9330, -0.9330],
          ...,
          [ 1.2980,  1.2980,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
          [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
          [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897]],
 
         [[-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
          [-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
          [-1.4580, -1.4580, -1.4580,  ..., -0.3725, -0.3550, -0.3550],
          ...,
          [ 1.0980,  1.0980,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
          [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
          [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206]],
 
         [[-1.6555, -1.6555, -1.6384,  ..., -0.9877, -0.9705, -0.9705],
          [-1.6555, -1.6555,

In [80]:
data_loader.dataset[0][1].bbox

tensor([[380.9447,  64.7477, 855.8875, 413.8505],
        [451.7142, 289.8131, 760.7495, 653.2710]])

In [168]:
torch.all(data_loader.dataset[0][0].to(device) == t_image)

tensor(True, device='cuda:0')

In [108]:
torch.all(data_loader.dataset[0][1].bbox == t_target[0].bbox)

tensor(True)

#### Run inference on loaded image and boxes

In [92]:
# input is t_image and t_target
@torch.no_grad()
def run_inference_with_image_and_boxes(t_image_list, box_list, model, cfg):
    """
    image_list: ImageList
    box_list: List[BoxList]
    """
    images = to_image_list(image_list)
    assert 'vl' not in cfg.MODEL.SWINT.VERSION, 'Only support vision inputs now'
    visual_features = model.backbone(image_list.tensors)    
    query_feats=model.pooler(visual_features, box_list) # num_boxes, num_channels, pooler_size, pooler_size
    query_feats=query_feats[None, ...] # 1, num_boxes, num_channels, pooler_size, pooler_size
    query_feats = query_feats.mean(dim=[-2,-1]).permute(1, 0, 2) # num_boxes, num_scales, num_channels
    labels=torch.cat([t.get_field('labels') for t in box_list])
    assert len(labels)==len(query_feats)
    max_query_number = cfg.VISION_QUERY.MAX_QUERY_NUMBER
    return query_feats, labels

In [169]:
new_query_feats, new_labels = run_inference_with_image_and_boxes(t_image_list, t_target, model, cfg)

In [171]:
new_labels

tensor([ 461, 1139], device='cuda:0')

In [170]:
new_query_feats.shape

torch.Size([2, 1, 256])

##### check inference results

In [172]:
# original
query_feats.shape

torch.Size([36, 1, 256])

In [173]:
torch.all(query_feats[:2] == new_query_feats)

tensor(False, device='cuda:0')

In [174]:
query_feats[0, 0, :10]

tensor([-1.8489,  0.4200,  0.4687, -0.8887, -1.1715,  1.8230,  0.6481,  0.3095,
        -0.7059,  0.7495], device='cuda:0')

In [175]:
new_query_feats[0, 0, :10]

tensor([-1.8383,  0.4312,  0.4734, -0.8855, -1.1696,  1.8102,  0.6440,  0.3118,
        -0.7052,  0.7440], device='cuda:0')

In [ ]:
labels

In [176]:
new_query_feats2, new_labels2 = run_inference_with_image_and_boxes(t_image_list, t_target, model, cfg)

In [177]:
new_query_feats2[0, 0, :10]

tensor([-1.8383,  0.4312,  0.4734, -0.8855, -1.1696,  1.8102,  0.6440,  0.3118,
        -0.7052,  0.7440], device='cuda:0')

In [194]:
t_image_list.tensors.shape

torch.Size([1, 1, 32, 800, 1196])

In [188]:
# note the batched version input images will have paddingm so there will be some differences in output
torch.all(t_image_list.tensors == images.tensors[0:1, :, :t_image_list.tensors.shape[2], :t_image_list.tensors.shape[3]])

tensor(True, device='cuda:0')

In [179]:
t_image_list.tensors

tensor([[[[-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
          [-1.3164, -1.2990, -1.2816,  ..., -0.9156, -0.9330, -0.9330],
          [-1.2990, -1.2990, -1.2990,  ..., -0.9156, -0.9330, -0.9330],
          ...,
          [ 1.2980,  1.2980,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
          [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897],
          [ 1.2805,  1.2805,  1.2980,  ...,  1.4897,  1.4897,  1.4897]],

         [[-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
          [-1.4755, -1.4580, -1.4405,  ..., -0.3725, -0.3550, -0.3550],
          [-1.4580, -1.4580, -1.4580,  ..., -0.3725, -0.3550, -0.3550],
          ...,
          [ 1.0980,  1.0980,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
          [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206],
          [ 1.0805,  1.0805,  1.0980,  ...,  1.2206,  1.2206,  1.2206]],

         [[-1.6555, -1.6555, -1.6384,  ..., -0.9877, -0.9705, -0.9705],
          [-1.6555, -1.6555, -

In [183]:
# note the zero paddings
images.tensors[0:1]

tensor([[[[-1.3164, -1.2990, -1.2816,  ...,  0.0000,  0.0000,  0.0000],
          [-1.3164, -1.2990, -1.2816,  ...,  0.0000,  0.0000,  0.0000],
          [-1.2990, -1.2990, -1.2990,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.4755, -1.4580, -1.4405,  ...,  0.0000,  0.0000,  0.0000],
          [-1.4755, -1.4580, -1.4405,  ...,  0.0000,  0.0000,  0.0000],
          [-1.4580, -1.4580, -1.4580,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.6555, -1.6555, -1.6384,  ...,  0.0000,  0.0000,  0.0000],
          [-1.6555, -1.6555, -